## Regression

In [9]:
# Import Libraries
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [10]:
# Read the dataset
df = pd.read_csv("../dataset/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Data Preprocessing

In [11]:
# Drop the columns that are not needed
df = df.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

In [12]:
# Encode the categorical variables
label_encoder = LabelEncoder()
df["Gender"] = label_encoder.fit_transform(df["Gender"])

# One-hot encode the 'Geography' column
ohe_geo = OneHotEncoder(handle_unknown="ignore")
geo_encoded = ohe_geo.fit_transform(df[["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(["Geography"]))

# Combine the encoded columns with the original dataframe
df = pd.concat([df.drop("Geography", axis=1), geo_encoded_df], axis=1)

In [13]:
# Split the dataset into features and target variable
X = df.drop("EstimatedSalary", axis=1)
y = df["EstimatedSalary"]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
# Save the label encoder and one-hot encoder
pkl_files = ["label_encoder.pkl", "ohe_geo.pkl", "scaler.pkl"]

for pkl in pkl_files:
  with open(pkl, "wb") as file:
    if pkl == "label_encoder.pkl":
      pickle.dump(label_encoder, file)
    elif pkl == "ohe_geo.pkl":
      pickle.dump(ohe_geo, file)
    else:
      pickle.dump(scaler, file)

### ANN with Regression

In [18]:
# Import libraries for model training
import tensorflow as tf
import datetime

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [16]:
# Model building
model = Sequential([
  Dense(64, activation="relu", input_shape=(X_train.shape[1],)), # HL1 connected to input layer
  Dense(32, activation="relu"), ## HL2 connected to HL1
  Dense(1) # Output layer for binary classification (linear activation for regression output)
])

# Compile the model
model.compile(optimizer="adam", loss="mean_absolute_error", metrics=["mae"])

# Dispkay the model summary
model.summary()

c:\MySkill\Data Science\Projects\ann-dl-project\venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Set up tensorboard
log_dir = "regression/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Set up early stopping
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [20]:
# Train the model
history = model.fit(
  X_train, y_train,
  validation_data=(X_test, y_test),
  epochs=100,
  callbacks=[early_stop, tensorboard_callback],
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 99464.2734 - mae: 99464.2734 - val_loss: 98504.7266 - val_mae: 98504.7266
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 98811.1641 - mae: 98811.1641 - val_loss: 96877.5781 - val_mae: 96877.5781
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 98370.1328 - mae: 98370.1328 - val_loss: 92710.7266 - val_mae: 92710.7266
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 91880.7969 - mae: 91880.7969 - val_loss: 85802.6016 - val_mae: 85802.6016
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 84778.9844 - mae: 84778.9844 - val_loss: 77036.0547 - val_mae: 77036.0547
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 76920.7266 - mae: 76920.7266 - val_loss: 68077.7969 - val_mae: 68077.7969
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 66633.7188 - mae: 66633.7188 - val_loss: 60487.6797 - val_mae: 60487.6797
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - 

In [22]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
%tensorboard --logdir regression/logs/fit

Reusing TensorBoard on port 6006 (pid 24816), started 0:00:28 ago. (Use '!kill 24816' to kill it.)

In [24]:
# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51124.2695 - mae: 51124.2695  
Test Loss: 50256.6328125, Test MAE: 50256.6328125


In [25]:
# Save the regression model
model.save("regression_model.h5")